<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/post_ollama_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
import json
import os
import time
from tqdm import tqdm

In [25]:
#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [32]:
#un thread demone avvia il server locale di ollama
import subprocess
import threading
t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
t.start()

Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [33]:
#un altro thread demone avvia llama3
!ollama pull llama3
t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", "llama3"]),daemon=True)
t2.start()

[GIN] 2024/07/01 - 13:52:15 | 200 |      26.309µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ [GIN] 2024/07/01 - 13:52:15 | 200 |  538.759769ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [34]:
moderator_instructions = [
'You are a decentralized social-network instance\'s moderator. Your role is to check if utent\'s post respects the instance rules. You must assign each post a score from 0 (if the post violate completely the instance\'s rules) to 10 (if the post respects all the instance\'s rules). Your response must be only the score number and anything else.' ,
'You are a decentralized social-network instance\'s owner. Your role is to check if utent\'s post respects the instance rules. In order to classify posts you have to assign each post a score from 0(if the post violate completely the instance\'s rules) to 10(if the post respects all the instance\'s rules). You must be very strict in your evaluations. Please pay a lot of attention and let\'s think step by step. Your response must be only the score number and anything else.'
]


In [35]:
def ask_to_llama(instruction,rules, prompt):    
    full_context = instruction
    for i, rule in enumerate(rules, start=1):
        full_context += f"{i}. {rule['text']}\n"
    
    full_prompt = full_context + prompt
    
    response = requests.post('http://localhost:11434/api/generate', 
                             data=json.dumps({'model': 'llama3', 'prompt': full_prompt, 'stream': False}), 
                             headers={'Content-Type': 'application/json'})
    
    return response.json()['response']

[GIN] 2024/07/01 - 13:52:16 | 200 |      31.723µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/07/01 - 13:52:16 | 200 |   25.887782ms |       127.0.0.1 | POST     "/api/show"
[GIN] 2024/07/01 - 13:52:16 | 200 |    1.284151ms |       127.0.0.1 | POST     "/api/generate"


In [36]:
def process_json_files(instances, start, end, save_dir):
    
    start_time = time.time()
    istanze = os.listdir(instances)
    already_proc = os.listdir(save_dir)
    
    
    for i in tqdm(range(start, end)):
        filename = istanze[i]
        if not f'{filename[:-5]}_scores.json' in already_proc:
            filepath = os.path.join(instances, filename)
            with open(filepath, 'r') as file:
                data = json.load(file)
                instance_name = data['name']
                results = {'name': instance_name, 'rules': data['rules'], 'records':[]}
                rules = data['rules']
                records = data['records']
                records_en = [record for record in records if record['language'] == 'en']
                        
                if len(records_en)==0:
                    print(f"Per l'istanza {instance_name} non ci sono post in inglese")
                    filt_recorods = []
                    
                if len(records_en)>200: #se len>200 prendiamo solo testa e coda
                    records_ordinati = sorted(records_en, key=lambda x: x['favourites'])
                    primi_100 = records_ordinati[:100]
                    ultimi_100 = records_ordinati[-100:]
                    filt_records = primi_100 + ultimi_100
                    
                else: #altrimenti li prendiamo tutti
                    filt_records = records_en
                
                print("Numero di post: ",len(filt_records))
                i=0
                for record in filt_records:
                    record_text = record['text']
                    rec = {'id': record['id'], 'text': record_text, 'tests':[]}
                    i+=1
                    for instruction in moderator_instructions:
                        response = ask_to_llama(instruction,rules,record_text)
                        rec['tests'].append({'instruction': instruction, 'score': response})
                        #break #così vediamo cosa fa con le altre instanze
                    results['records'].append(rec)
                    print(f"Processati {i} post")
                print(f'instance {instance_name} successfully processed\n')
                if not os.path.isdir(save_dir):
                    os.mkdir(save_dir)
                with open(os.path.join(save_dir, f'{instance_name}_scores.json'),'w') as f:
                    json.dump(results, f, indent = 4)
                    #break 
        break
    end_time = time.time() 
    duration = end_time - start_time
    print("DURATA: ", duration)

In [37]:
instances = '/kaggle/input/instance-json/results'
process_json_files(instances,0,1,'/kaggle/working/')
#len(os.listdir(instances))//num_profili


  0%|          | 0/1 [00:00<?, ?it/s]

Numero di post:  200
[GIN] 2024/07/01 - 13:52:23 | 200 |  2.218799042s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/01 - 13:52:25 | 200 |  2.199394864s |       127.0.0.1 | POST     "/api/generate"
Processati 1 post
[GIN] 2024/07/01 - 13:52:27 | 200 |  2.232819551s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/01 - 13:52:30 | 200 |  2.241168502s |       127.0.0.1 | POST     "/api/generate"
Processati 2 post
[GIN] 2024/07/01 - 13:52:32 | 200 |  2.234079957s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/01 - 13:52:34 | 200 |   2.24006156s |       127.0.0.1 | POST     "/api/generate"
Processati 3 post
[GIN] 2024/07/01 - 13:52:36 | 200 |  2.232627115s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/01 - 13:52:39 | 200 |  2.238958865s |       127.0.0.1 | POST     "/api/generate"
Processati 4 post
[GIN] 2024/07/01 - 13:52:41 | 200 |  2.233905978s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/01 - 13:52:43 | 200 |  2.239732929s | 

  0%|          | 0/1 [14:56<?, ?it/s]

[GIN] 2024/07/01 - 14:07:17 | 200 |  2.242045222s |       127.0.0.1 | POST     "/api/generate"
Processati 200 post
instance tchafia.be successfully processed

DURATA:  896.7094864845276
